# GroupBy机制

In [1]:
import numpy as np
import pandas as pd

split-apply-combine（拆分-应用-合并）

In [3]:
df = pd.DataFrame({'key1':['a','a','b','b','a'],'key2':['one','two','one','two','one'],
                  'data1':np.random.randn(5),'data2':np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,0.348814,0.227657
1,a,two,0.042775,-0.213586
2,b,one,0.523572,-1.099279
3,b,two,0.846972,-1.346748
4,a,one,-1.846461,2.035594


想要按 key1进行分组，并计算data1列的平均值。我们用的是：访问data1，并根据key1调用groupby：

In [6]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [7]:
grouped.mean()

key1
a   -0.484957
b    0.685272
Name: data1, dtype: float64

如果我们一次性传入多个数组的列表：

In [11]:
means = df['data1'].groupby([df['key1'],df['key2']]).mean()
means                #传入多个数组的列表，根据这些数组进行分组

key1  key2
a     one    -0.748824
      two     0.042775
b     one     0.523572
      two     0.846972
Name: data1, dtype: float64

这里，通过两个键对数据进行了分组，得到的Series具有一个层次化索引（由唯一的键对组成）：

In [13]:
means.unstack()  #解除层次化索引，默认操作的是最内层，保留最外层

key2,one,two
key1,,
a,-0.748824,0.042775
b,0.523572,0.846972


在这个例子中，分组键均为Series。实际上，分组键可以是任何长度的数组：

In [15]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])
df['data1'].groupby([states,years]).mean()

California  2005    0.042775
            2006    0.523572
Ohio        2005    0.597893
            2006   -1.846461
Name: data1, dtype: float64

通常，分组信息位于相同的要处理的DataFrame中。这里还可以将列名（可以是字符串、数字或其他python对象）用作分组键：

In [17]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-0.484957,0.683222
b,0.685272,-1.223014


In [18]:
df.groupby(['key1','key2']).mean()

data1     data2
key1 key2                    
a    one  -0.748824  1.131626
     two   0.042775 -0.213586
b    one   0.523572 -1.099279
     two   0.846972 -1.346748

groupby的size方法。返回一个包含分组大小的Series：

In [21]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

注意，任何分组关键词中的缺失值，都会被从结果中除去。

# 对分组进行迭代

groupby对象支持迭代，可以产生一组二元元组（由分组名和数据块组成）。

In [25]:
for name,group in df.groupby('key1'):
    print(name)     #分组名  a，b
    print(group)    # 数据块

a
  key1 key2     data1     data2
0    a  one  0.348814  0.227657
1    a  two  0.042775 -0.213586
4    a  one -1.846461  2.035594
b
  key1 key2     data1     data2
2    b  one  0.523572 -1.099279
3    b  two  0.846972 -1.346748


对于多重键的情况，元组的第一个元素将会是由键值组成的元组！：

In [28]:
for (k1,k2),group in df.groupby(['key1','key2']):
    print((k1,k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.348814  0.227657
4    a  one -1.846461  2.035594
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.042775 -0.213586
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.523572 -1.099279
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.846972 -1.346748


做成一个字典

In [33]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  0.348814  0.227657
 1    a  two  0.042775 -0.213586
 4    a  one -1.846461  2.035594, 'b':   key1 key2     data1     data2
 2    b  one  0.523572 -1.099279
 3    b  two  0.846972 -1.346748}

In [34]:
pieces['b']

,key1,key2,data1,data2
2,b,one,0.523572,-1.099279
3,b,two,0.846972,-1.346748


groupby默认是在axis=0上进行分组的，通过设置也可以在其他轴上进行分组。也可以根据dtype对列进行分组：

In [35]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [42]:
grouped = df.groupby(df.dtypes,axis=1)
for dtype,group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.348814  0.227657
1  0.042775 -0.213586
2  0.523572 -1.099279
3  0.846972 -1.346748
4 -1.846461  2.035594
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


# 选取一列或列的子集

对于由DataFrame产生的groupby对象，如果用一个（单个字符串）或一组（字符串数组）列名对其进行索引。就能实现选取部分列进行聚合的目的。也就是说：

df.groupby('key1')['data1']  ==  df['data1'].groupby(df['key1'])     #  Series                                       
df.groupby('key1')[['data2']] == df[['data2']].groupby(df['key1'])   #  DataFrame

In [49]:
df.groupby(['key1','key2'])[['data2']].mean()     # [ [   ] ]  只计算data2的平均值 并以DataFrame 返回结果 

data2
key1 key2          
a    one   1.131626
     two  -0.213586
b    one  -1.099279
     two  -1.346748

这种索引操作返回的对象是一个已分组的DataFrame（如果传入的是列表或数组）或已分组的Series（如果传入的是标量形式的单个列名）：

In [51]:
df.groupby(['key1','key2'])['data1'].mean()  

key1  key2
a     one    -0.748824
      two     0.042775
b     one     0.523572
      two     0.846972
Name: data1, dtype: float64

# 通过字典或Series进行分组

除数组外，分组信息还可以其他形式存在：

In [64]:
people = pd.DataFrame(np.random.rand(5,5),columns = ['a','b','c','d','e'],
                      index=['Joe','Steve','Wes','Jim','Travis'])
people

,a,b,c,d,e
Joe,0.056446,0.409482,0.025569,0.542811,0.467240
Steve,0.460001,0.472672,0.649102,0.563745,0.542918
Wes,0.057294,0.038629,0.710820,0.872896,0.233615
Jim,0.693070,0.470950,0.263637,0.017136,0.123907
Travis,0.184847,0.417572,0.844149,0.983387,0.637559


In [65]:
people.iloc[2:3,1:3] = np.nan   # Add a few NA values   iloc[2:3,1:3]=iloc[2:3,[1,2]]
people

,a,b,c,d,e
Joe,0.056446,0.409482,0.025569,0.542811,0.467240
Steve,0.460001,0.472672,0.649102,0.563745,0.542918
Wes,0.057294,NaN,NaN,0.872896,0.233615
Jim,0.693070,0.470950,0.263637,0.017136,0.123907
Travis,0.184847,0.417572,0.844149,0.983387,0.637559


In [66]:
mapping = {'a':'red','b':'red','c':'blue','d':'blue','e':'red','f':'orange'} #分组关系，可以纯在未使用的分组键
by_columns = people.groupby(mapping,axis=1)
by_columns.sum()

,blue,red
Joe,0.568380,0.933169
Steve,1.212847,1.475591
Wes,0.872896,0.290909
Jim,0.280773,1.287927
Travis,1.827536,1.239978


Series也有同样的功能，它可以被看做一个固定大小的映射：

In [67]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [68]:
people.groupby(map_series,axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


# 通过函数进行分组 

In [70]:
people.groupby(len).sum()   # 被当做分组键的函数都会在各个索引值上被调用一次  

,a,b,c,d,e
3,0.806810,0.880432,0.289206,1.432844,0.824762
5,0.460001,0.472672,0.649102,0.563745,0.542918
6,0.184847,0.417572,0.844149,0.983387,0.637559


将函数跟数组、列表、字典、Series混用，任何东西在内部都会被转换为数组：

In [71]:
key_list=['one','one','one','two','two']   #  key_list 对应len的索引值（即行标签）
people.groupby([len,key_list]).min()

a         b         c         d         e
3 one  0.056446  0.409482  0.025569  0.542811  0.233615
  two  0.693070  0.470950  0.263637  0.017136  0.123907
5 one  0.460001  0.472672  0.649102  0.563745  0.542918
6 two  0.184847  0.417572  0.844149  0.983387  0.637559

# 根据索引级别分组

层次化索引数据集最方便的地方就在于它能够根据轴索引的一个级别进行聚合：

In [74]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],[1,3,5,1,3]],
                                  names = ['city','tenot'])
hier_df = pd.DataFrame(np.random.randn(4,5),columns = columns)
hier_df

city         US                            JP          
tenot         1         3         5         1         3
0     -1.064243  0.875396  1.177255 -1.320083 -1.596476
1     -1.491973  0.393402 -1.729958 -0.721938  0.957760
2     -1.064263 -0.457552  0.290419  0.560384 -0.999896
3      0.203009  0.034886  1.008247  1.332189 -0.598282

要根据级别分组，使用level关键字传递级别序号或名字：

In [77]:
hier_df.groupby(level = 'city',axis = 1).count()   # 根据列索引分组，则axis=1

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 数据聚合

聚合指的是任何能从数组产生标量值的数据转换过程。如：mean（）、count（）、min、sun等。

In [78]:
df

,key1,key2,data1,data2
0,a,one,0.348814,0.227657
1,a,two,0.042775,-0.213586
2,b,one,0.523572,-1.099279
3,b,two,0.846972,-1.346748
4,a,one,-1.846461,2.035594


In [83]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)  #quantil()  计算样本分位数

key1
a    0.287606
b    0.814632
Name: data1, dtype: float64

如果要使用自己的聚合函数，将其传入aggregate或agg方法：

In [84]:
def peak_to_peak(arr):
    return arr.max()-arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.195276,2.249181
b,0.323400,0.247469


describe也可以用在这里，即使严格来讲并不是聚合运算：

In [85]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.484957  1.188985 -1.846461 -0.901843  0.042775  0.195795   
b      2.0  0.685272  0.228678  0.523572  0.604422  0.685272  0.766122   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.348814   3.0  0.683222  1.191788 -0.213586  0.007035  0.227657   
b     0.846972   2.0 -1.223014  0.174987 -1.346748 -1.284881 -1.223014   

                          
           75%       max  
key1                      
a     1.131626  2.035594  
b    -1.161147 -1.099279

# 面向列的多函数应用

In [88]:
tips = pd.read_csv('examples\\tips.csv')
tips

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4
5,25.29,4.71,No,Sun,Dinner,4
6,8.77,2.00,No,Sun,Dinner,2
7,26.88,3.12,No,Sun,Dinner,4
8,15.04,1.96,No,Sun,Dinner,2
9,14.78,3.23,No,Sun,Dinner,2


In [89]:
tips['tip_pct'] = tips['tip']/tips['total_bill'] # add tip percentage of total bill
tips[:6]    

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


希望对不同的列使用不同的聚合函数，或一次应用多个函数。

In [90]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')   # 对于常见的描述统计（如 mean ，std，max），可以将函数名以字符串的形式传入：

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

如果传入一组函数或函数名（列表的形式），得到的DataFrame的列就会以相应的函数命名：

In [92]:
grouped_pct.agg(['mean','std',peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

传入一个由（name，function）元组组成的列表，则各元组的第一个元素就会被用作DataFrame的列名（可以将这种二元元组列表看做一个有序序列）：

In [99]:
grouped_pct.agg([('平均值','mean'),('标准差',np.std)])   # np.std = 'std'

平均值       标准差
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

对于DataFrame，有更多的选择，可以定义一组应用于全部列的一组函数，或不同的列应用不同的函数。假设想要对tip_pct和total_bill列计算三个统计信息：

In [100]:
funcs = ['mean','count','max']
result = grouped['tip_pct','total_bill'].agg(funcs)
result

tip_pct                 total_bill             
                 mean count       max       mean count    max
day  smoker                                                  
Fri  No      0.151650     4  0.187735  18.420000     4  22.75
     Yes     0.174783    15  0.263480  16.813333    15  40.17
Sat  No      0.158048    45  0.291990  19.661778    45  48.33
     Yes     0.147906    42  0.325733  21.276667    42  50.81
Sun  No      0.160113    57  0.252672  20.506667    57  48.17
     Yes     0.187250    19  0.710345  24.120000    19  45.35
Thur No      0.160298    45  0.266312  17.113111    45  41.19
     Yes     0.163863    17  0.241255  19.190588    17  43.11

对一个列或不同的列应用不同的函数，具体的方法是想agg传入一个从列名映射到函数的字典!!!!!!!!!!!：

In [101]:
grouped.agg({'tip':np.max,'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [102]:
grouped.agg({'tip_pct':['min','max','mean','std'],'size':'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

只有将多个函数！！应用到至少一列时，DataFrame才会拥有层次化的列。

# 以“没有行索引”的形式返回聚合数据

向groupby传入as_index = False 可以将分组键组成的索引重新返回到数组中，即重新生成默认的行索引,不以分组键为行索引.

In [103]:
tips.groupby(['day','smoker'],as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 3.apply：一般性的“拆分-应用-合并”

最通用的Groupby方法是apply。  先拆分，然后各自应用传入的函数，最后尝试合并到一起

假设想要根据分组选出最高的5个tip_pct值。首先，先编写一个选取指定列具有最大值的行的函数：

In [107]:
def top5(df,n=5,column = 'tip_pct'):
    return  df.sort_values(by = column)[-n:]
top5(tips,n=6)  # n=6 选取6个

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


现在，对smoker分组并用该函数调用apply，得到：

In [109]:
tips.groupby('smoker').apply(top5)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

top函数在DataFrame的各个片段（smoker的Yes和No）调用，然后结果由pandas.concat组装到一起，并以分组名称进行了标记。于是，最终结果就有了一个层次化索引，其内层索引值来自原DataFrame。

In [112]:
tips.groupby(['smoker','day']).apply(top5,n=1,column = 'total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [113]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [123]:
result.unstack()

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

# 禁止分组键

分组键会和原始对象的索引共同构成结果对象中的层次化索引，在groupby中传入group_keys=False可禁止该效果：

In [124]:
tips.groupby('smoker').apply(top5)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [125]:
tips.groupby('smoker',group_keys=False).apply(top5)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


# 分位数和桶分析

In [127]:
frame = pd.DataFrame({'data1':np.random.randn(1000),'data2':np.random.randn(1000)})
frame

,data1,data2
0,-0.618708,1.081782
1,0.953645,1.694535
2,0.047367,0.248987
3,-0.611998,1.942950
4,-1.897375,-0.441706
5,-1.286321,0.226822
6,-0.072323,-0.219601
7,-0.424592,-0.234964
8,-0.402572,-0.241528
9,-1.470184,-1.703366


In [128]:
quartiles = pd.cut(frame.data1,4)
quartiles

0      (-1.642, -0.0706]
1       (-0.0706, 1.501]
2       (-0.0706, 1.501]
3      (-1.642, -0.0706]
4       (-3.219, -1.642]
5      (-1.642, -0.0706]
6      (-1.642, -0.0706]
7      (-1.642, -0.0706]
8      (-1.642, -0.0706]
9      (-1.642, -0.0706]
10     (-1.642, -0.0706]
11     (-1.642, -0.0706]
12     (-1.642, -0.0706]
13      (-0.0706, 1.501]
14      (-0.0706, 1.501]
15     (-1.642, -0.0706]
16      (-0.0706, 1.501]
17      (-0.0706, 1.501]
18     (-1.642, -0.0706]
19      (-0.0706, 1.501]
20      (-3.219, -1.642]
21     (-1.642, -0.0706]
22      (-0.0706, 1.501]
23     (-1.642, -0.0706]
24     (-1.642, -0.0706]
25      (-0.0706, 1.501]
26      (-0.0706, 1.501]
27      (-0.0706, 1.501]
28        (1.501, 3.072]
29     (-1.642, -0.0706]
             ...        
970     (-0.0706, 1.501]
971     (-0.0706, 1.501]
972     (-3.219, -1.642]
973    (-1.642, -0.0706]
974    (-1.642, -0.0706]
975     (-0.0706, 1.501]
976     (-0.0706, 1.501]
977     (-3.219, -1.642]
978    (-1.642, -0.0706]


In [129]:
quartiles[:10]

0    (-1.642, -0.0706]
1     (-0.0706, 1.501]
2     (-0.0706, 1.501]
3    (-1.642, -0.0706]
4     (-3.219, -1.642]
5    (-1.642, -0.0706]
6    (-1.642, -0.0706]
7    (-1.642, -0.0706]
8    (-1.642, -0.0706]
9    (-1.642, -0.0706]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.219, -1.642] < (-1.642, -0.0706] < (-0.0706, 1.501] < (1.501, 3.072]]

由cut返回的Categorical对象可以直接传递到groupby。

In [133]:
def get_stats(group):
    return {'min':group.min(),'max':group.max(),'count':group.count(),'mean':group.mean()}
grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-3.219, -1.642]",59.0,2.199203,-0.102658,-1.912483
"(-1.642, -0.0706]",464.0,3.386913,0.046735,-2.895508
"(-0.0706, 1.501]",420.0,2.696153,-0.095450,-2.911307
"(1.501, 3.072]",57.0,2.350015,-0.143899,-2.884848


这些都是长度相等的桶。要根据样本分位数得到大小相等的桶，使用qcut即可。传入labels=False即可只获取分位数的编号。

In [141]:
grouping = pd.qcut(frame.data1,10,labels = False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()   # cut 等宽分隔 即按连续数据的大小分到各个桶里，每个桶里样本量可能不同，但是，每个桶相当于一个等长的区间
                                    # qcut ：等频分隔    每个桶里的样本数量是一定的

,count,max,mean,min
data1,,,,
0,100.0,3.037853,-0.056059,-1.935284
1,100.0,3.386913,0.260745,-1.937709
2,100.0,2.403893,-0.088600,-2.895508
3,100.0,2.576245,-0.009633,-2.266124
4,100.0,1.946574,0.010292,-1.840191
5,100.0,2.450165,-0.060002,-2.911307
6,100.0,1.945384,-0.170266,-2.722755
7,100.0,2.696153,-0.120937,-2.424321
8,100.0,2.490868,0.070635,-2.059915


# 示例：用特定于分组的值填充缺失值

In [143]:
s = pd.Series(np.random.randn(6))
s

0    0.580486
1    0.292179
2   -0.648132
3    1.382316
4   -3.040428
5   -0.243813
dtype: float64

In [145]:
s[::2]=np.nan
s

0         NaN
1    0.292179
2         NaN
3    1.382316
4         NaN
5   -0.243813
dtype: float64

In [146]:
s.fillna(s.mean())  # 求 mean（）  分母不算NaN的个数

0    0.476894
1    0.292179
2    0.476894
3    1.382316
4    0.476894
5   -0.243813
dtype: float64

假设需要对不同的分组填充不同的值。一种方法是将数据分组，并使用apply和一个能够对各数据块调用fillna的函数。

In [149]:
states = ['Ohio','NY','Ver','Feor','Ore','Nevada','Cali','Idaho']
group_key = ['East']*4+['West']*4# ['East']*4 产生一个列表，包括了['East']中元素的四个拷贝。将这些列表串联起来
data = pd.Series(np.random.randn(8),index = states)
data

Ohio     -0.122540
NY       -0.997218
Ver       0.782500
Feor      0.691113
Ore       0.085879
Nevada    0.803755
Cali     -0.585805
Idaho     1.092246
dtype: float64

In [150]:
data[['Ver','Nevada','Idaho']] = np.nan
data

Ohio     -0.122540
NY       -0.997218
Ver            NaN
Feor      0.691113
Ore       0.085879
Nevada         NaN
Cali     -0.585805
Idaho          NaN
dtype: float64

In [152]:
data.groupby(group_key).mean()

East   -0.142882
West   -0.249963
dtype: float64

In [156]:
fill_mean = lambda g:g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio     -0.122540
NY       -0.997218
Ver      -0.142882
Feor      0.691113
Ore       0.085879
Nevada   -0.249963
Cali     -0.585805
Idaho    -0.249963
dtype: float64

也可以在代码中预定义各组的填充值。由于分组具有一个name属性，所以：

In [157]:
fill_values = {'East':0.5,'West':-1}
fill_func = lambda g:g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio     -0.122540
NY       -0.997218
Ver       0.500000
Feor      0.691113
Ore       0.085879
Nevada   -1.000000
Cali     -0.585805
Idaho    -1.000000
dtype: float64

# 随机采样和排列

In [164]:
suits = ['H','S','C','D']
card_val = (list(range(1,11))+[10]*3)*4
base_names = ['A']+list(range(2,11))+['J','K','Q']
cards = []
for suit in suits:
    cards.extend(str(num)+suit for num in base_names)
deck = pd.Series(card_val,index = cards)
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [165]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [166]:
def draw(deck,n=5):
    return deck.sample(n)
draw(deck)

2C     2
7C     7
KC    10
7H     7
3C     3
dtype: int64

In [171]:
get_suits = lambda card: card[-1] # 取花色
deck.groupby(get_suits).apply(draw,n=2)  # 每种花色随机抽取两张

C  JC     10
   KC     10
D  3D      3
   AD      1
H  9H      9
   JH     10
S  10S    10
   7S      7
dtype: int64

In [172]:
deck.groupby(get_suits,group_keys=False).apply(draw,n=2)

AC      1
4C      4
5D      5
AD      1
4H      4
5H      5
3S      3
10S    10
dtype: int64

# 实例：分组加权平均数和相关系数